# Get started 
(last updated: 2024/04/17 zensvi=v0.11.0)

To use `zensvi` in a project, run the following command. Please make sure to install a compatiable version of `PyTorch` and `torchvision` separately before running the command.

See the [documentation](https://pytorch.org/get-started/locally/) for more information on how to install PyTorch.

In [ ]:
%pip install --upgrade zensvi # if zen-svi is not installed

import zensvi

print(zensvi.__version__)

from zensvi.download import MLYDownloader
from zensvi.cv import Segmenter, ClassifierPlaces365, get_low_level_features
from zensvi.transform import ImageTransformer
from zensvi.visualization import plot_image, plot_map

# How to download Street View Images
## Mapillary

In [ ]:
mly_api_key = "YOUR_OWN_MLY_API_KEY"  # please register your own Mapillary API key at https://www.mapillary.com/dashboard/developers
# additional keyword arguments for the Mapillary API
kwarg = {
    "fields": ["all"],  # A list of options, either as [‘all’], or a list of fields.
    "zoom": 14,  # The zoom level of the tiles to obtain, defaults to 14
    "radius": 50,  # The radius of the images obtained from a center center
    "image_type": "both",  # The tile image_type to be obtained, either as ‘flat’, ‘pano’ (panoramic), or ‘both’.
    "min_captured_at": "2016",  # The min date. Format from ‘YYYY’, to ‘YYYY-MM-DDTHH:MM:SS’
    "max_captured_at": "2020",  # The max date. Format from ‘YYYY’, to ‘YYYY-MM-DDTHH:MM:SS’
    "org_id": None,  # The organization id, ID of the organization this image (or sets of images) belong to. It can be absent. Thus, default is -1 (None)
}
mly_downloader = MLYDownloader(
    mly_api_key,  # Mapillary API key
    log_path=None,  # path to the log file
)
mly_downloader.download_svi(
    "path/to/output",  # output directory
    path_pid=None,  # if you already have a list of panorama IDs, you can specify the path to the file here
    lat=None,
    lon=None,  # latitude and longitude of the location to download
    input_csv_file="",  # path to the input CSV file containing the location information
    input_shp_file="",  # path to the input shapefile containing the location information
    input_place_name="",  # name of the location to download
    id_columns=None,  # column name of the ID in the input CSV file or shapefile
    buffer=0,  # buffer size in meters around the input location
    update_pids=False,  # if True, the list of panorama IDs will be updated. If False, the list of panorama IDs will be loaded from the path_pid file
    resolution=1024,  # resolution of the image
    cropped=False,  # if True, only the upper half of the image is saved
    network_type=network_type,  # network type for OSMnx
    custom_filter=custom_filter,  # custom filter for OSMnx
    batch_size=1000,  # batch size for downloading images
    start_date=None,  # start date for downloading images (YYYY-MM-DD)
    end_date=None,  # end date for downloading images (YYYY-MM-DD)
    metadata_only=False,  # if True, only metadata is downloaded
    use_cache=True,  # if True, the cache is used
    **kwarg
)

# How to run segmentation
## Cityscapes

In [ ]:
# initialize the segmenter
segmenter = Segmenter()

# set arguments
dir_input = "path/to/input"
dir_image_output = "path/to/image_output"
dir_summary_output = "path/to/summary_output"
save_image_options = "segmented_image blend_image"  # segmented_image (colored image), blend_image (blended image)
pixel_ratio_save_format = "csv json"
segmenter.segment(
    dir_input,
    dir_image_output=dir_image_output,
    dir_summary_output=dir_summary_output,
    save_image_options=save_image_options,
    pixel_ratio_save_format=pixel_ratio_save_format,
)

## Mapillary Vistas

In [ ]:
# initialize the segmenter
dataset = "mapillary"  # this can be either "mapillary" or "cityscapes"
task = "panoptic"  # this can be either "semantic" or "panoptic"
segmenter = Segmenter(dataset=dataset, task=task)

# set arguments
dir_input = "path/to/input"
dir_image_output = "path/to/image_output"
dir_summary_output = "path/to/summary_output"
save_image_options = "segmented_image blend_image"
pixel_ratio_save_format = "csv json"
csv_format = "long"  # "long" or "wide"
segmenter.segment(
    dir_input,
    dir_image_output=dir_image_output,
    dir_summary_output=dir_summary_output,
    save_image_options=save_image_options,
    pixel_ratio_save_format=pixel_ratio_save_format,
    csv_format=csv_format,
)

# How to run classification
## Places365

In [ ]:
# initialize the classifier
classifier = ClassifierPlaces365(
    device="cpu",  # device to use (either "cpu" or "gpu")
)

# set arguments
classifier = ClassifierPlaces365()
dir_input = "path/to/input"
dir_image_output = "path/to/image_output"
dir_summary_output = "path/to/summary_output"
csv_format = "long"  # "long" or "wide"
classifier.classify(
    dir_input,
    dir_image_output=dir_image_output,
    dir_summary_output=dir_summary_output,
    csv_format=csv_format,
)

# How to get low-level features

In [ ]:
dir_input = "path/to/input"
dir_image_output = "path/to/image_output"
dir_summary_output = "path/to/summary_output"
csv_format = "long"  # "long" or "wide"
get_low_level_features(
    dir_input,
    dir_image_output=dir_image_output,
    dir_summary_output=dir_summary_output,
    save_format="json csv",
    csv_format=csv_format,
)

# How to transform images: from panorama to perspective and fisheye

In [ ]:
dir_input = "path/to/input"
dir_output = "path/to/output"
image_transformer = ImageTransformer(dir_input=dir_input, dir_output=dir_output)
image_transformer.transform_images(
    style_list="perspective equidistant_fisheye orthographic_fisheye stereographic_fisheye equisolid_fisheye",  # list of projection styles in the form of a string separated by a space
    FOV=90,  # field of view
    theta=120,  # angle of view (horizontal)
    phi=0,  # angle of view (vertical)
    aspects=(9, 16),  # aspect ratio
    show_size=100,
)  # size of the image to show (i.e. scale factor)

# How to visualize the results
## Point map

In [ ]:
dir_input = "path/to/input"
path_output = "path/to/output.png"  # output file path
path_pid = "path/to/pid"  # path to the panorama ID file with latitude and longitude
csv_file_pattern = "pixel_ratios.csv"  # pattern of the CSV files that contain the pixel ratios (or any other variable to plot)
variable = "vegetation"  # variable to plot (e.g. vegetation, building, sky, etc.). This should be the column name in the CSV file. If None, count of the number of images is plotted
plot_type = "point"  # plot type (either "point", "line", or "hexagon")
fig, ax = plot_map(
    path_pid,
    dir_input=dir_input,
    csv_file_pattern=csv_file_pattern,
    variable_name=variable,
    plot_type=plot_type,
    path_output=path_output,
    resolution=13,
    cmap="viridis",
    legend=True,
    title="Point Map",
    legend_title="Vegetation",
    dark_mode=False,
)

## Line map

In [ ]:
dir_input = "path/to/input"
path_output = "path/to/output.png"  # output file path
path_pid = "path/to/pid"  # path to the panorama ID file with latitude and longitude
csv_file_pattern = "pixel_ratios.csv"  # pattern of the CSV files that contain the pixel ratios (or any other variable to plot)
variable = "vegetation"  # variable to plot (e.g. vegetation, building, sky, etc.). This should be the column name in the CSV file. If None, count of the number of images is plotted
plot_type = "line"  # plot type (either "point", "line", or "hexagon")
fig, ax = plot_map(
    path_pid,
    dir_input=dir_input,
    csv_file_pattern=csv_file_pattern,
    variable_name=variable,
    plot_type=plot_type,
    path_output=path_output,
    resolution=13,
    cmap="viridis",
    legend=True,
    title="Point Map",
    legend_title="Vegetation",
    dark_mode=False,
)

## Hexagon map

In [ ]:
dir_input = "path/to/input"
path_output = "path/to/output.png"  # output file path
path_pid = "path/to/pid"  # path to the panorama ID file with latitude and longitude
csv_file_pattern = "pixel_ratios.csv"  # pattern of the CSV files that contain the pixel ratios (or any other variable to plot)
variable = "vegetation"  # variable to plot (e.g. vegetation, building, sky, etc.). This should be the column name in the CSV file. If None, count of the number of images is plotted
plot_type = "hexagon"  # plot type (either "point", "line", or "hexagon")
fig, ax = plot_map(
    path_pid,
    dir_input=dir_input,
    csv_file_pattern=csv_file_pattern,
    variable_name=variable,
    plot_type=plot_type,
    path_output=path_output,
    resolution=13,
    cmap="viridis",
    legend=True,
    title="Point Map",
    legend_title="Vegetation",
    dark_mode=False,
)

## Plot images as grid

In [ ]:
dir_image_input = "path/to/input"
path_output = "path/to/output.png"  # output file path
image_file_pattern = "*.png"  # pattern of the image files to plot
dir_csv_input = "path/to/csv"  # directory of the CSV files
csv_file_pattern = "pixel_ratios.csv"  # pattern of the CSV files
sort_by = "random"  # sort the images by either "random" or names of variables in the CSV files (e.g. "vegetation")
fig, ax = plot_image(
    dir_image_input,
    4,  # number of rows
    5,  # number of columns
    dir_csv_input=dir_csv_input,  # directory of the CSV files
    csv_file_pattern=csv_file_pattern,  # pattern of the CSV files
    sort_by=sort_by,  # sort the images by either "random" or names of variables in the CSV files (e.g. "vegetation")
    title="Image Grid",
    path_output=path_output,
    dark_mode=False,  # if True, the background is dark
    random_seed=123,
)